# Correlating NYT Data with Market Data

### Loading in Data

In [23]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
import string
from dateutil.relativedelta import relativedelta
import numpy as np
import yfinance as yf

In [61]:
spy = yf.download("SPY")
nyt = pd.read_csv("./Data/NYTDataCleaned.csv", encoding = "ISO-8859-1")

[*********************100%***********************]  1 of 1 completed


In [63]:
# Match nyt data to the data we have on SPY and combine them into one dataframe
# spy = spy.iloc[6969:] # Remove this time
spy.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-10-01,337.690002,338.739990,335.010010,337.040009,337.040009,88698700
2020-10-02,331.700012,337.010010,331.190002,333.839996,333.839996,89431100
2020-10-05,336.059998,339.959991,336.010010,339.760010,339.760010,45713100
2020-10-06,339.910004,342.170013,334.380005,334.929993,334.929993,90128900
2020-10-07,338.119995,341.630005,338.089996,340.760010,340.760010,56999600
